# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao construir a **pontuação de crédito** de um cliente em potencial. A **pontuação de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

Esse projeto tem como objetivo a elaboração de relatório de risco para a divisão de emprestimos de um banco, demonstrando os impactos de fatores como estado civil e número de de filhos, por exemplo, quando falamos de inadimplência. Deverá ser construído indicador chamado **pontuação de crédito**, que será usado para avaliação do cliente acerca da capacidade de pagamento dos devedores.

In [1]:
import pandas as pd

cse = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [2]:
cse

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


In [3]:
cse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
cse.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


O arquivo de dados tem 21525 linhas e 12 colunas, tendo sido evidenciada a presença de duas colunas com cerca de 10% dados igualmente em ambas, **days_employed** e **total_income** . Existem também valores negativos e distorcidos. Serão feitas análise e correção das distorções identificadas.

In [5]:
cse.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Só constam valores ausentes nas colunas 'days_employed' e 'total_income', as demais não. Será necessário fazer análises e comparações entre essas colunas e as que não apresentam valores ausentes para criar hipóteses e posterior correção dos mesmos.

Antes de tratar os valores ausentes, será feita analise e correção dos dados duplicados existentes.

In [6]:
cse.duplicated().sum()

54

In [7]:
cse = cse.drop_duplicates()

In [8]:
cse.duplicated().sum()

0

In [9]:
cse[cse['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [10]:
cse.isna().sum()

children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

Os valores ausentes são simétricos, conforme resultado abaixodo método shape aplicado.

In [11]:
cse[(cse['days_employed'].isnull()) & (cse['total_income'].isnull())].shape

(2120, 12)

**Conclusão intermediária**

O número de linhas na tabela é exatamente o mesmo para ambas colunas, 2120, 'days_employed'e 'total_income', indicando que os dados ausentes podem ter a mesma origem.

* O percentual de valores ausentes é de 9,873%, valor já calculado com exclusão de duplicadas.

As próximas tratativas serão a exclusão de **valores duplicados**, tratar e corrigir os **dados ausentes** e eventuais **distorções de valores** das colunas.

In [12]:
[cse['days_employed'].isnull().sum()/len(cse)*100]

[9.873783242513156]

In [13]:
cse_null = cse[(cse['days_employed'].isnull()) & (cse['total_income'].isnull())]

cse_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2120 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2120 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2120 non-null   int64  
 3   education         2120 non-null   object 
 4   education_id      2120 non-null   int64  
 5   family_status     2120 non-null   object 
 6   family_status_id  2120 non-null   int64  
 7   gender            2120 non-null   object 
 8   income_type       2120 non-null   object 
 9   debt              2120 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2120 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 215.3+ KB


In [14]:
cse_null['income_type'].value_counts(normalize=True)

employee         0.508019
business         0.237264
retiree          0.185849
civil servant    0.068396
entrepreneur     0.000472
Name: income_type, dtype: float64

**Possíveis motivos para valores ausentes nos dados**

50,8% dos valores ausentes referem-se a pessoas com emprego formal e 23,7% com seu próprio negócio. 18,6% de aposentados. Isso demonstra que trata-se de dado possivelmente não coletado do cliente, ou seja, sem padrão definido.


In [15]:
cse['income_type'].value_counts(normalize=True)

employee                       0.516557
business                       0.236598
retiree                        0.178706
civil servant                  0.067859
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000047
paternity / maternity leave    0.000047
Name: income_type, dtype: float64

**Conclusão intermediária**

A distribuição da tabela completa e sem valores nulos seguem a mesma distribuição. 

In [16]:
cse.corr()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,1.000000,-0.138354,-0.173885,-0.012477,-0.091435,0.018336,0.018220
days_employed,-0.138354,1.000000,0.582643,0.080565,0.005726,-0.047110,-0.136648
dob_years,-0.173885,0.582643,1.000000,0.066885,-0.069684,-0.069379,-0.052911
education_id,-0.012477,0.080565,0.066885,1.000000,0.007990,0.052964,-0.178885
family_status_id,-0.091435,0.005726,-0.069684,0.007990,1.000000,0.020389,-0.009147
debt,0.018336,-0.047110,-0.069379,0.052964,0.020389,1.000000,-0.012475
total_income,0.018220,-0.136648,-0.052911,-0.178885,-0.009147,-0.012475,1.000000


**Conclusão intermediária**

De acordo com os dados analisados e tratados, não foi evidenciado padrão para determinar que os valores poderia estar ausentes de forma não acidental.

**Conclusões**

Nenhum padrão foi encontrado, conforme evidenciado no % de ausentes proporcionais em levantamento realizado com toda a tabela e sem os valores ausentes da coluna tipo de emprego 'income_type'.

Serão tratados os valores ausentes primeiramente verificando se existem correlações entre colunas que subsidiarão tomada de decisão do tipo de tratativa a ser dada para substituição e/ou eventual exclusão de dados.

Antes de tratar os valores ausentes, analisaremos com grandes distorções, valores negativos ou com erro.

## Transformação de dados

Será feita a análise por coluna para possível identificação dos dados ausentes apresentados e também valores incorretos e/ou com distorções.

As duplicadas já foram removidas anteriormente.

In [17]:
cse_edu = cse['education'].unique()
cse_edu

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [106]:
cse['education'] = cse['education'].str.lower()
cse['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [19]:
cse['education'].value_counts(normalize=True)

secondary education    0.707373
bachelor's degree      0.244562
some college           0.034651
primary education      0.013134
graduate degree        0.000279
Name: education, dtype: float64

coluna `children`

In [20]:
cse['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [21]:
cse['children'].value_counts(normalize=True)

 0     0.657026
 1     0.223977
 2     0.095571
 3     0.015370
 20    0.003540
-1     0.002189
 4     0.001910
 5     0.000419
Name: children, dtype: float64

Na coluna 'children' foram identificados dois problemas: valor negativo e um valor irreal de número de filhos. Será feita aplicação da coluna para que demonstre valores absolutos, corrigindo o valor negativo para positivo e alterando *20* para *2* no caso do valor distorcido.

In [107]:
cse['children'] = cse['children'].replace(20,2)

In [108]:
cse['children'] = cse['children'].replace(-1,1)

In [24]:
cse['children'].value_counts(normalize=True)

0    0.657026
1    0.226166
2    0.099110
3    0.015370
4    0.001910
5    0.000419
Name: children, dtype: float64

coluna `days_employed`

In [25]:
cse['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [26]:
(cse['days_employed'] < 0).sum()

15906

Os dados da coluna 'days_employed' estão em 75% das linhas, ou seja, uma alta incidência. Precisaremos explorar esses dados para posteriormente realizar a correção, pois não podemos descartar essas linhas problemáticas.

In [27]:
cse[cse['days_employed']<0]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [109]:
cse['days_employed'] = cse['days_employed'].abs()
cse['days_employed'].describe()

count    21454.000000
mean     13865.457626
std      24208.813737
min         24.141633
25%       1012.345888
50%       2358.542802
75%       6073.803283
max      66914.728907
Name: days_employed, dtype: float64

In [29]:
cse_days = cse[cse['days_employed'] < 27365]
mean_days_employed = cse_days['days_employed'].mean()
mean_days_employed 

2353.0159319988766

In [30]:
median_days_employed = cse_days['days_employed'].median()
median_days_employed 

1630.0193809778218

In [31]:
pivot_days_genero = cse_days.pivot_table(index = ['dob_years','gender'], values='days_employed', aggfunc ='mean')
pivot_days_genero

days_employed
dob_years gender               
0         F         2540.659172
          M         1572.160272
19        F          622.060062
          M          672.404832
20        F          619.050333
...                         ...
72        F         6603.920060
73        F         3429.205485
74        F         6682.867814
          M         1729.632531
75        F         1678.969771

[114 rows x 1 columns]

In [32]:
cse_days_2 = cse[cse['days_employed'] >= 27365]
cse_days_2

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


In [110]:
mean_days_employed = cse['days_employed'].mean()
cse.loc[cse['days_employed'] >= 27365, 'days_employed'] = mean_days_employed
cse['days_employed'].describe()

count    21454.000000
mean      4668.785561
std       5083.982104
min         24.141633
25%       1012.345888
50%       2358.542802
75%       6073.803283
max      23022.470366
Name: days_employed, dtype: float64

In [111]:
cse_days.groupby(['dob_years','gender'])['days_employed'].transform('mean')
cse['days_employed'] = cse['days_employed'].fillna(cse.groupby(['dob_years','gender'])['days_employed'].transform('mean'))

In [35]:
cse_days['days_employed'].value_counts(normalize=True)

142.276217      0.000063
4595.317076     0.000063
366.029564      0.000063
7375.209318     0.000063
799.071855      0.000063
                  ...   
2456.531274     0.000063
3132.849226     0.000063
2139.352596     0.000063
10727.562704    0.000063
1636.419775     0.000063
Name: days_employed, Length: 15906, dtype: float64

In [36]:
cse_days['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

In [37]:
cse['days_employed'].head(15)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4     66914.728907
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
10     4171.483647
11      792.701887
12    44256.003683
13     1846.641941
14     1844.956182
Name: days_employed, dtype: float64

coluna 'dob_years'

In [38]:
cse['dob_years'].value_counts(normalize=True)

35    0.028690
40    0.028271
41    0.028224
34    0.027991
38    0.027805
42    0.027758
33    0.027060
39    0.026641
31    0.026035
36    0.025802
44    0.025383
29    0.025337
30    0.025057
48    0.025010
37    0.024964
50    0.023893
43    0.023846
32    0.023706
49    0.023660
28    0.023427
45    0.023148
27    0.022961
52    0.022542
56    0.022542
47    0.022216
54    0.022169
46    0.022030
53    0.021378
57    0.021238
58    0.021238
51    0.020865
59    0.020632
55    0.020632
26    0.019002
60    0.017419
25    0.016627
61    0.016487
62    0.016254
63    0.012529
24    0.012296
64    0.012203
23    0.011783
65    0.009035
22    0.008523
66    0.008477
67    0.007778
21    0.005170
0     0.004704
68    0.004611
69    0.003959
70    0.003027
71    0.002701
20    0.002375
72    0.001537
19    0.000652
73    0.000373
74    0.000279
75    0.000047
Name: dob_years, dtype: float64

Conforme observado, o % de valores ausentes não segue nenhum padrão ou apresenta evidência do porquê da existencia deles. Serão calculadas as medias e medianas de acordo com o gênero, e aplicaremos o metodo mais adequado para subsituição e correção dos valores ausentes, bem como valores incorretos.

In [39]:
cse[cse['dob_years']<18]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,66914.728907,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,66914.728907,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,18634.176605,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,66914.728907,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,66914.728907,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


In [40]:
cse['dob_years'].median()


42.0

In [41]:
cse['dob_years'].mean()

43.279074099948765

In [42]:
pivot_days_genero = cse.pivot_table(index = ['days_employed','gender'], values='dob_years', aggfunc ='mean')
pivot_days_genero

,,dob_years
days_employed,gender,
24.141633,F,31.000000
24.240695,M,32.000000
30.195337,M,47.000000
33.520665,M,43.000000
34.701045,F,31.000000
...,...,...
61849.307381,F,71.000000
62724.598665,F,69.000000
64047.639066,M,68.000000


In [43]:
pivot_days_genero = cse.pivot_table(index = ['days_employed','gender'], values='dob_years', aggfunc ='mean')
pivot_days_genero

,,dob_years
days_employed,gender,
24.141633,F,31.000000
24.240695,M,32.000000
30.195337,M,47.000000
33.520665,M,43.000000
34.701045,F,31.000000
...,...,...
61849.307381,F,71.000000
62724.598665,F,69.000000
64047.639066,M,68.000000


In [112]:
cse['dob_years'].replace(0, cse['dob_years'].median() , inplace=True)
cse['dob_years'].describe()

count    21454.000000
mean        43.468957
std         12.213881
min         19.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [45]:
cse['dob_years'].value_counts(normalize=True).sort_index()

19.0    0.000652
20.0    0.002375
21.0    0.005170
22.0    0.008523
23.0    0.011783
24.0    0.012296
25.0    0.016627
26.0    0.019002
27.0    0.022961
28.0    0.023427
29.0    0.025337
30.0    0.025057
31.0    0.026035
32.0    0.023706
33.0    0.027060
34.0    0.027991
35.0    0.028690
36.0    0.025802
37.0    0.024964
38.0    0.027805
39.0    0.026641
40.0    0.028271
41.0    0.028224
42.0    0.032462
43.0    0.023846
44.0    0.025383
45.0    0.023148
46.0    0.022030
47.0    0.022216
48.0    0.025010
49.0    0.023660
50.0    0.023893
51.0    0.020865
52.0    0.022542
53.0    0.021378
54.0    0.022169
55.0    0.020632
56.0    0.022542
57.0    0.021238
58.0    0.021238
59.0    0.020632
60.0    0.017419
61.0    0.016487
62.0    0.016254
63.0    0.012529
64.0    0.012203
65.0    0.009035
66.0    0.008477
67.0    0.007778
68.0    0.004611
69.0    0.003959
70.0    0.003027
71.0    0.002701
72.0    0.001537
73.0    0.000373
74.0    0.000279
75.0    0.000047
Name: dob_years, dtype: float64

coluna `family_status`

In [46]:
cse['family_status'].value_counts(dropna=False)


married              12344
civil partnership     4163
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

In [47]:
cse['family_status'].describe()

count       21471
unique          5
top       married
freq        12344
Name: family_status, dtype: object

In [48]:
cse.groupby('dob_years')['family_status'].value_counts()

dob_years  family_status    
19.0       unmarried             7
           civil partnership     4
           married               3
20.0       unmarried            26
           civil partnership    12
                                ..
73.0       civil partnership     1
74.0       married               4
           civil partnership     1
           widow / widower       1
75.0       widow / widower       1
Name: family_status, Length: 267, dtype: int64

coluna `gender`

In [49]:
cse['gender'].value_counts(dropna=False)

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [50]:
index = cse.loc[cse['gender'] == 'XNA']
print(index)

       children  days_employed  dob_years     education  education_id  \
10701         0    2358.600502       24.0  some college             2   

           family_status  family_status_id gender income_type  debt  \
10701  civil partnership                 1    XNA    business     0   

       total_income          purpose  
10701     32624.825  buy real estate  


In [51]:
cse_null['gender'].value_counts(normalize=True)

F    0.67783
M    0.32217
Name: gender, dtype: float64

In [52]:
cse['gender'].describe()

count     21471
unique        3
top           F
freq      14189
Name: gender, dtype: object

coluna `income_type`

In [53]:
cse['income_type'].value_counts(dropna=False)

employee                       11091
business                        5080
retiree                         3837
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [54]:
cse['income_type'].describe()

count        21471
unique           8
top       employee
freq         11091
Name: income_type, dtype: object

In [55]:
cse.groupby('dob_years')['income_type'].describe()

,count,unique,top,freq
dob_years,,,,
19.0,14,3,business,7
20.0,51,3,employee,33
21.0,111,3,employee,69
22.0,183,5,employee,111
23.0,253,3,employee,175
24.0,264,4,employee,162
25.0,357,3,employee,222
26.0,408,4,employee,249
27.0,493,5,employee,309


Confirmando abaixo a remoção feita anteriormente de valores duplicados

In [56]:
cse.duplicated().sum()
cse = cse.drop_duplicates()

In [57]:
cse.duplicated().sum()

0

In [58]:
cse.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


As correções reduziram significamente os valores ausentes. Agora faremos os demais ajustes dos valores ainda não realizados.


# Trabalhando com valores ausentes

Serão utilizados adicionalmente bibliotecas com dados sobre educação/escolaridade.

In [59]:
cse


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,66914.728907,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,66914.728907,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [60]:
education_dict=cse[['education_id', 'education']]
education_dict=education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [61]:
education_dict=cse['education'].str.lower()

### Restaurar valores ausentes em `total_income`

A coluna 'total_income' apresenta ainda valores ausentes. Serão feitos calculos de media e mediana de acordo com a idade e gênero para tomada de decisão do método mais adequado para substituir os valores ausentes ainda não corrigidos.

In [62]:
def cat_age(age):
    if age <= 18:
        return 'young'
    if age <=60:
        return 'adult'
    return 'eldery'

In [63]:
# Teste se a função funciona
print(cat_age(2))
print(cat_age(40))
print(cat_age(80))

young
adult
eldery


In [64]:
cse['age_category'] = cse['dob_years'].apply(cat_age)

In [65]:
cse.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,66914.728907,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [66]:
cse_sem_nulos = cse[cse['days_employed'].notna()]
cse_sem_nulos

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,66914.728907,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21521,0,66914.728907,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,eldery
21522,1,2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21523,3,3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


In [67]:
cse_sem_nulos['total_income'].mean()

26787.568354658677

In [68]:
cse_sem_nulos['total_income'].median()

23202.87

In [69]:
cse_sem_nulos.pivot_table(index='gender', values='total_income',aggfunc = 'mean')

,total_income
gender,
F,24655.604757
M,30907.144369
XNA,32624.825000


In [70]:
cse_sem_nulos.pivot_table(index=['education','gender'], values='total_income',aggfunc = 'median')

total_income
education           gender              
bachelor's degree   F         26063.4715
                    M         32675.8355
graduate degree     F         29345.3940
                    M         25161.5835
primary education   F         17223.9615
                    M         21204.0860
secondary education F         20101.2700
                    M         25435.5815
some college        F         22836.0820
                    M         29973.6640
                    XNA       32624.8250

In [71]:
pivot_education_genero = cse_sem_nulos.pivot_table(index = ['education','gender'], values='total_income', aggfunc ='mean')
pivot_education_genero

total_income
education           gender              
bachelor's degree   F       30306.441576
                    M       38981.070503
graduate degree     F       29345.394000
                    M       27267.340000
primary education   F       19118.479588
                    M       23798.931664
secondary education F       22671.099805
                    M       28296.294264
some college        F       26470.312199
                    M       33209.842210
                    XNA     32624.825000

In [72]:
pivot_education_genero = cse_sem_nulos.pivot_table(index = ['education','gender'], values='total_income', aggfunc ='median')
pivot_education_genero

total_income
education           gender              
bachelor's degree   F         26063.4715
                    M         32675.8355
graduate degree     F         29345.3940
                    M         25161.5835
primary education   F         17223.9615
                    M         21204.0860
secondary education F         20101.2700
                    M         25435.5815
some college        F         22836.0820
                    M         29973.6640
                    XNA       32624.8250

In [73]:
pivot_education_genero['total_income']["bachelor's degree"]['M']

32675.8355

Serão utilizadas as médias calculadas de acordo com a coluna gênero e education.

In [74]:
def fill_total_income(row):
    
    total_income = row['total_income']
    educacao = row['education']
    genero = row['gender']
    
    if pd.isna(total_income):
        return pivot_education_genero['total_income'][educacao][genero]
    return total_income
        

In [75]:
cse['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

In [76]:
cse['total_income'] = cse.apply(fill_total_income, axis=1)

In [77]:
cse['total_income'].isna().sum()

0

In [78]:
cse.groupby(['education','gender'])['total_income'].transform('median')
cse['total_income'] = cse['total_income'].fillna(cse.groupby(['education','gender'])['total_income'].transform('median'))
print(cse.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028       42.0    bachelor's degree             0   
1         1    4024.803754       36.0  secondary education             1   
2         0    5623.422610       33.0  secondary education             1   
3         3    4124.747207       32.0  secondary education             1   
4         0   66914.728907       53.0  secondary education             1   
5         0     926.185831       27.0    bachelor's degree             0   
6         0    2879.202052       43.0    bachelor's degree             0   
7         0     152.779569       50.0  secondary education             1   
8         2    6929.865299       35.0    bachelor's degree             0   
9         0    2188.756445       41.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [79]:
cse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_category      21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


###  Restaurar valores em `days_employed`

[Pense nos parâmetros que podem ajudá-lo a restaurar os valores ausentes nesta coluna. Eventualmente, você desejará descobrir se deve usar valores médios ou medianos para substituir valores ausentes. Você provavelmente fará uma pesquisa semelhante à que fez ao restaurar dados em uma coluna anterior.]

In [80]:
cse['days_employed'].median()

2358.5428021913895

In [81]:
cse['days_employed'].mean()

13865.457625616455

Os valores ausentes da coluna days_employed já foram restaurados acima, quando estavamos tratando por coluna. E também explicado que foram utilizados as médias para realizar a substituição.

In [82]:
cse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_category      21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB


## Categorização de dados

[Para responder às perguntas e testar as hipóteses, você vai querer trabalhar com dados categorizados. Veja as perguntas que lhe foram colocadas e que você deve responder. Pense em quais dados precisarão ser categorizados para responder a essas perguntas. Abaixo, você encontrará um modelo por meio do qual poderá trabalhar à sua maneira ao categorizar os dados. O primeiro processamento passo a passo abrange os dados de texto; o segundo aborda os dados numéricos que precisam ser categorizados. Você pode usar ambas ou nenhuma das instruções sugeridas - você decide.]

[Independentemente de como você decidir abordar a categorização, certifique-se de fornecer uma explicação clara do por quê você tomou sua decisão. Lembre-se: este é o seu trabalho e você toma todas as decisões nele.]


In [83]:
print(cse.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_category      21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.3+ MB
None


In [84]:
cse['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [85]:
cse['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [86]:
cse['debt'].unique()

array([0, 1])

In [87]:
cse['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [88]:
cse['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [89]:
cse['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [90]:
cse['total_income'].unique()

array([40620.102, 17932.802, 23341.752, ..., 14347.61 , 39054.888,
       13127.587])

In [91]:
def debt_group(debt):
    
    if debt == 0:
        return 'Adimplente'
    if debt == 1:
        return 'Histórico Inadimplencia'

print(debt_group(0))
print(debt_group(1))

Adimplente
Histórico Inadimplencia


In [92]:
for index, row in cse.iterrows():
    if isinstance(row['purpose'], str) and 'education' in row['purpose'] or 'educated' in row['purpose'] or 'university' in row['purpose']:
        cse.at[index, 'purpose_categoria'] = 'education'
    elif isinstance(row['purpose'], str) and ('car' in row['purpose'] or 'cars' in row['purpose']):
        cse.at[index, 'purpose_categoria'] = 'cars'
    elif isinstance(row['purpose'], str) and 'wedding' in row['purpose']:
        cse.at[index, 'purpose_categoria'] = 'wedding'
    elif isinstance(row['purpose'], str) and ('house' in row['purpose'] or 'housing' in row['purpose'] or 'property' in row['purpose'] or 'real estate' in row['purpose']):
        cse.at[index, 'purpose_categoria'] = 'house'
        
print(cse['purpose_categoria'].head(15))
print(cse.head())
print(cse.info())

0         house
1          cars
2         house
3     education
4       wedding
5         house
6         house
7     education
8       wedding
9         house
10        house
11        house
12      wedding
13         cars
14        house
Name: purpose_categoria, dtype: object
   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028       42.0    bachelor's degree             0   
1         1    4024.803754       36.0  secondary education             1   
2         0    5623.422610       33.0  secondary education             1   
3         3    4124.747207       32.0  secondary education             1   
4         0   66914.728907       53.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married 

In [93]:
cse.groupby(['children', 'dob_years', 'total_income'])['debt'].median()

children  dob_years  total_income
0         19.0       9459.8510       0.0
                     10963.8570      0.0
                     12125.9860      0.0
                     12929.9440      0.0
                     14575.7170      0.0
                                    ... 
5         36.0       26953.7480      0.0
          37.0       41071.7360      0.0
          38.0       34007.2590      0.0
          42.0       25435.5815      0.0
          59.0       43050.9360      0.0
Name: debt, Length: 19866, dtype: float64

In [94]:
cse.groupby(['children', 'dob_years', 'total_income'])['debt'].sum()

children  dob_years  total_income
0         19.0       9459.8510       0
                     10963.8570      0
                     12125.9860      0
                     12929.9440      0
                     14575.7170      0
                                    ..
5         36.0       26953.7480      0
          37.0       41071.7360      0
          38.0       34007.2590      0
          42.0       25435.5815      0
          59.0       43050.9360      0
Name: debt, Length: 19866, dtype: int64

In [95]:
cse.groupby(['children', 'dob_years', 'debt'])['total_income'].mean()

children  dob_years  debt
0         19.0       0       17801.687115
                     1       14934.901000
          20.0       0       20129.247014
                     1       17737.316000
          21.0       0       23996.384076
                                 ...     
5         36.0       0       17378.705500
          37.0       0       41071.736000
          38.0       0       34007.259000
          42.0       0       25435.581500
          59.0       0       43050.936000
Name: total_income, Length: 364, dtype: float64

In [96]:
cse.groupby(['children', 'dob_years', 'debt'])['total_income'].median()

children  dob_years  debt
0         19.0       0       16588.2370
                     1       14934.9010
          20.0       0       17257.2770
                     1       19975.1040
          21.0       0       21361.7300
                                ...    
5         36.0       0       17378.7055
          37.0       0       41071.7360
          38.0       0       34007.2590
          42.0       0       25435.5815
          59.0       0       43050.9360
Name: total_income, Length: 364, dtype: float64

In [97]:
cse.groupby(['children', 'dob_years', 'debt'])['total_income'].head()

0        40620.1020
1        17932.8020
2        23341.7520
3        42820.5680
4        25378.5720
            ...    
21333    21125.1220
21390    25435.5815
21412    42408.8810
21456    18446.6750
21523    39054.8880
Name: total_income, Length: 1475, dtype: float64

In [98]:
print('DOB YEARS')

dob_years = cse['dob_years']

def categorizar_dob_years(dob_years):
    if dob_years <= 18:
        return 'young'
    elif dob_years <=60:
        return 'adult'
    else:
        return 'eldery'
  
cse['cat_dob_years'] = dob_years.apply(categorizar_dob_years)
print(cse[['dob_years', 'cat_dob_years']].head(15))    


print('COLUNA CHILDREN')

children = cse['children']

def categorizar_children(children):
    if children <= 1:
        return 'até 1 filho'
    elif children <= 3:
        return 'até 3 filhos'
    else:
        return 'até 5 filhos'
    
cse['cat_children'] = children.apply(categorizar_children)
print(cse[['children', 'cat_children']].head(15)) 


print('TOTAL INCOME')

total_income = cse['total_income']

def categorizar_total_income(income):
    if income <= 30000:
        return 'até 30 mil'
    elif income <= 50000:
        return '30 a 50 mil'
    else:
        return 'acima de 50 mil'
    
cse['cat_total_income'] = total_income.apply(categorizar_total_income)
print(cse[['total_income', 'cat_total_income']].head(15))

DOB YEARS
    dob_years cat_dob_years
0        42.0         adult
1        36.0         adult
2        33.0         adult
3        32.0         adult
4        53.0         adult
5        27.0         adult
6        43.0         adult
7        50.0         adult
8        35.0         adult
9        41.0         adult
10       36.0         adult
11       40.0         adult
12       65.0        eldery
13       54.0         adult
14       56.0         adult
COLUNA CHILDREN
    children  cat_children
0          1   até 1 filho
1          1   até 1 filho
2          0   até 1 filho
3          3  até 3 filhos
4          0   até 1 filho
5          0   até 1 filho
6          0   até 1 filho
7          0   até 1 filho
8          2  até 3 filhos
9          0   até 1 filho
10         2  até 3 filhos
11         0   até 1 filho
12         0   até 1 filho
13         0   até 1 filho
14         0   até 1 filho
TOTAL INCOME
    total_income cat_total_income
0     40620.1020      30 a 50 mil
1     17932.8

In [99]:
cse['cat_dob_years'] = cse['dob_years']
print(cse.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028       42.0    bachelor's degree             0   
1         1    4024.803754       36.0  secondary education             1   
2         0    5623.422610       33.0  secondary education             1   
3         3    4124.747207       32.0  secondary education             1   
4         0   66914.728907       53.0  secondary education             1   
5         0     926.185831       27.0    bachelor's degree             0   
6         0    2879.202052       43.0    bachelor's degree             0   
7         0     152.779569       50.0  secondary education             1   
8         2    6929.865299       35.0    bachelor's degree             0   
9         0    2188.756445       41.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [100]:
print(cse[['dob_years', 'cat_dob_years']].value_counts())  
print(cse[['children', 'cat_children']].value_counts())  
print(cse[['total_income', 'cat_total_income']].value_counts())  

dob_years  cat_dob_years
42.0       42.0             697
35.0       35.0             616
40.0       40.0             607
41.0       41.0             605
34.0       34.0             601
38.0       38.0             597
33.0       33.0             581
39.0       39.0             572
31.0       31.0             559
36.0       36.0             554
44.0       44.0             545
29.0       29.0             544
30.0       30.0             537
48.0       48.0             536
37.0       37.0             536
50.0       50.0             513
43.0       43.0             512
32.0       32.0             509
49.0       49.0             508
28.0       28.0             503
45.0       45.0             496
27.0       27.0             493
52.0       52.0             484
56.0       56.0             483
47.0       47.0             477
54.0       54.0             476
46.0       46.0             472
53.0       53.0             459
57.0       57.0             456
58.0       58.0             454
51.0       51.0

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [101]:
pivot_table_children = cse.pivot_table(index = 'children', columns='debt', values='dob_years', aggfunc='count')
pivot_table_children

pivot_table_children['percent_1'] = pivot_table_children[1] / (pivot_table_children[1]+ pivot_table_children[0]) * 100
pivot_table_children.sort_values(by='percent_1', ascending=True)

debt,0,1,percent_1
children,,,
0,13028.0,1063.0,7.543822
3,303.0,27.0,8.181818
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Conclusão**

Cliente com e sem filhos, em todas as faixas de renda, apresentaram taxa de inadimplência entre 7,54 (sem filhos) a 9,75% (4 filhos).

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [102]:
pivot_table_family_status = cse.pivot_table(index = 'family_status', columns='debt', values='dob_years', aggfunc='count')
pivot_table_family_status

pivot_table_family_status['percent_2'] = pivot_table_family_status[1] / (pivot_table_family_status[1]+ pivot_table_family_status[0]) * 100
pivot_table_family_status.sort_values(by='percent_2', ascending=True)

debt,0,1,percent_2
family_status,,,
widow / widower,896,63,6.569343
divorced,1110,85,7.112971
married,11408,931,7.545182
civil partnership,3763,388,9.347145
unmarried,2536,274,9.750890


**Conclusão**

Clientes de todos os tipos de status familiar apresentaram de inadimplência entre 6,57 (viúvos e viúvas) a 9,75% (solteiros).

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [103]:
pivot_table_family_status = cse.pivot_table(index = 'total_income', columns='debt', values='dob_years', aggfunc='count')
pivot_table_family_status

pivot_table_family_status['percent_3'] = pivot_table_family_status[1] / (pivot_table_family_status[1]+ pivot_table_family_status[0]) * 100
pivot_table_family_status.sort_values(by='percent_3', ascending=True).head(15)

debt,0,1,percent_3
total_income,,,
29973.6640,26.0,1.0,3.703704
32675.8355,161.0,8.0,4.733728
26063.4715,347.0,18.0,4.931507
22836.0820,41.0,3.0,6.818182
20101.2700,923.0,77.0,7.700000
25435.5815,419.0,61.0,12.708333
17223.9615,13.0,2.0,13.333333
21204.0860,6.0,1.0,14.285714
3306.7620,NaN,1.0,NaN


**Conclusão**

Não foi evidenciado padrão de acordo com a faixa de renda e status familiar

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [104]:
for index, row in cse.iterrows():
    if isinstance(row['purpose'], str) and 'education' in row['purpose'] or 'educated' in row['purpose'] or 'university' in row['purpose']:
        cse.at[index, 'purpose_categoria'] = 'education'
    elif isinstance(row['purpose'], str) and ('car' in row['purpose'] or 'cars' in row['purpose']):
        cse.at[index, 'purpose_categoria'] = 'cars'
    elif isinstance(row['purpose'], str) and 'wedding' in row['purpose']:
        cse.at[index, 'purpose_categoria'] = 'wedding'
    elif isinstance(row['purpose'], str) and ('house' in row['purpose'] or 'housing' in row['purpose'] or'property' in row['purpose'] or 'real estate' in row['purpose']):
        cse.at[index, 'purpose_categoria'] = 'house'

print(cse['purpose_categoria'])
print(cse.info())
print(cse.head(15))

0            house
1             cars
2            house
3        education
4          wedding
           ...    
21520        house
21521         cars
21522        house
21523         cars
21524         cars
Name: purpose_categoria, Length: 21454, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   children           21454 non-null  int64  
 1   days_employed      21454 non-null  float64
 2   dob_years          21454 non-null  float64
 3   education          21454 non-null  object 
 4   education_id       21454 non-null  int64  
 5   family_status      21454 non-null  object 
 6   family_status_id   21454 non-null  int64  
 7   gender             21454 non-null  object 
 8   income_type        21454 non-null  object 
 9   debt               21454 non-null  int64  
 10  total_income       21454 non-null  float64
 11 

In [105]:
pivot_table_deb_purpose_cat = cse.pivot_table(columns='purpose_categoria', values='debt', aggfunc='mean')*100
pivot_table_deb_purpose_cat

purpose_categoria,cars,education,house,wedding
debt,9.359034,9.220035,7.233373,8.003442


**Conclusão**

A taxa de inadimplência calculada por categoria de motivos para tomada de empréstimo indica:
7,23% tomaram crédito para custear compra, aluguel ou melhorias de residência
9,36% para compra, venda ou conserto de veículo
9,22% para investir em educação
8% para fins de casamento 

Conclui-se que clientes que tomam crédito para compra, venda ou conserto de veículo apresentam um risco maior de inadimplência, seguidos por educação, casamento e residência.

# Conclusão Geral 

Foram realizadas análise dos dados recebidos, com verificação dos valores ausentes, duplicados ou com erros e distorções. Foram feitos correção dos valores de acordo com média calculada entre gênero, idade, qantidade de filhos, critérios esses escolhidos de acordo com análise por coluna. Removidos também duplicados e corrigidos dados negativos ou incorretos.

De acordo com os cálculos de taxa de inadimplência e hipóteses analisadas, ficou evidenciado que clientes solteiros e com união estável, com 1 a 4 filhos e cujo motivo de tomada de crédito seja  para compra, venda ou conserto de veículo, são os que apresentam maior risco para o banco, de acordo com os parâmetros apresentados.